In [28]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write

In [29]:
from __future__ import annotations
import os
from typing import List, Dict
from huggingface_hub import InferenceClient
import pandas as pd
from google.colab import files
uploaded = files.upload()
import pandas as pd
import io
df = pd.read_csv("cases.csv").head(50)
df.head()


Saving cases.csv to cases (2).csv


,Unnamed: 0,id,text
0,0,1989_75,civil appellate jurisdiction civil appeal numb...
1,1,1959_102,civil appellate jurisdiction civil appeal numb...
2,2,1979_76,civil appellate jurisdiction civil appeal numb...
3,3,1991_124,civil appellate jurisdiction civil appeal numb...
4,4,1985_233,original jurisdiction writ petitions number. 8...


In [41]:
class LawyerAgent:

    def __init__(self,
                 name: str,
                 system_prompt: str,
                 model: str = "microsoft/Phi-3-mini-4k-instruct"):
        self.name = name
        self.system_prompt = system_prompt.strip()
        self.history: List[Dict[str, str]] = []  # list of {"role": ..., "content": ...}
        self.client = InferenceClient(
            model,
            token=os.getenv("HF_API_TOKEN")  # make sure this env‑var is set
        )

    def _format_prompt(self, user_msg: str, max_words: int = 3500) -> str:
        """
        Format prompt with truncation so total tokens stay within model limit.
        """
        messages = [{"role": "system", "content": self.system_prompt}]
        messages.extend(self.history)
        messages.append({"role": "user", "content": user_msg})

        prompt = ""
        for m in messages:
            prompt += f"<|{m['role']}|>\n{m['content']}\n"
        prompt += "<|assistant|>\n"

        # Truncate prompt if too long (approx by word count)
        words = prompt.split()
        if len(words) > max_words:
            prompt = " ".join(words[-max_words:])  # keep the last N words

        return prompt

    def respond(self, user_msg: str, **gen_kwargs) -> str:
        prompt = self._format_prompt(user_msg)
        completion = self.client.text_generation(
            prompt,
            max_new_tokens=512,
            temperature=0.5,
            do_sample=True,
            stream=False,
            **gen_kwargs
        )
        answer = completion.strip()

        # Limit history to avoid prompt growing too large
        self.history = self.history[-4:]

        # Update chat memory
        self.history.append({"role": "user", "content": user_msg})
        self.history.append({"role": "assistant", "content": answer})
        return answer


In [42]:
DEFENSE_SYSTEM = """You are **Alex Carter**, lead *defense counsel*.
Goals:
• Protect the constitutional rights of the defendant.
• Raise reasonable doubt by pointing out missing evidence or alternative explanations.
• Be respectful to the Court and to opposing counsel.
Style:
• Crisp, persuasive, grounded in precedent and facts provided.
• When citing precedent: give short case name + year (e.g., *Miranda v. Arizona* (1966)).
Ethics:
• Do not fabricate evidence; admit uncertainty when required.

"""
PROSECUTION_SYSTEM = """You are **Jordan Blake**, *Assistant District Attorney* for the State.
Goals:
• Present the strongest good‑faith case against the accused.
• Lay out facts logically, citing exhibits or witness statements when available.
• Anticipate and rebut common defense arguments.
Style:
• Formal but plain English; persuasive, with confident tone.
Ethics:
• Duty is to justice, not merely to win. Concede points when ethically required.
"""




JUDGE_CONDUCT = """
You are [Judge’s Name], Presiding *Judge* of the Court.
Goals:
• Ensure a fair, orderly, and impartial trial for all parties.
• Uphold and apply the law consistently, ruling on objections and motions with clarity.
• Protect the rights of the accused, the interests of the public, and the integrity of the proceedings.
Style:
• Formal, authoritative, and neutral in tone.
• Address all parties by appropriate titles; maintain control of the courtroom without unnecessary severity.
• Provide clear, measured explanations for rulings when appropriate.
Ethics:
• Remain impartial at all times, avoiding favoritism, prejudice, or impropriety.
• Ensure all parties have a fair opportunity to present their case.
• Intervene firmly but fairly to prevent misconduct, irrelevance, or undue influence on the jury.

"""

EXPERT_CONSULTANT = """
You are an **expert consultant** testifying in a criminal trial.
Goals:
• Provide clear, objective, and accurate explanations of specialized knowledge within your field.
• Assist the court and jury in understanding technical evidence and its relevance to the case.
• Maintain credibility by presenting findings without advocacy for either side.
Style:
• Precise, measured, and accessible — avoid excessive jargon.
• Focus on facts, methods, and conclusions supported by your expertise and recognized standards.
• Respond to cross-examination respectfully, clarifying or conceding limitations where appropriate.
Ethics:
• Testify to the full extent of your expertise; acknowledge the limits of your knowledge when applicable.
• Avoid partisanship — your duty is to the court’s understanding of the facts, not to the party that retained you.
• Do not speculate beyond your findings, and refrain from offering opinions on the defendant’s guilt or innocence.
"""







# agents

defense = LawyerAgent("Defense", DEFENSE_SYSTEM)
prosecution = LawyerAgent("Prosecution", PROSECUTION_SYSTEM)

judge = LawyerAgent("Judge", JUDGE_CONDUCT)

#expert = LawyerAgent("expert",EXPERT_CONSULTANT)



In [43]:
import pandas as pd

# ... (your agent initialization and other code) ...

predictions = []

for index, row in df.iterrows():
    case_background = row['text'][:150]  # Shortened input

    # ... (agent interactions, but without printing) ...

    # Judge's decision and verdict logic
    j_decision = judge.respond(f"Decision: {case_background}")
    verdict = 1 if "granted" in j_decision.lower() else 0

    predictions.append({'id': row['id'], 'label': verdict})

# Create DataFrame for predictions
predictions_df = pd.DataFrame(predictions)
predictions_df.to_csv('predictions.csv', index=False)
# Print or save the DataFrame in the desired format
print(predictions_df[['id', 'label']].to_string(index=False))
# Or save to a CSV:
# predictions_df[['ID', 'VERDICT']].to_csv('predictions.csv', index=False)
from google.colab import files
files.download('predictions.csv')

HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/hf-inference/models/microsoft/Phi-3-mini-4k-instruct (Request ID: Root=1-6803f247-1c2affcf093d13ed0c9a66d8;77697d6e-75f4-4db8-bd8a-21c5f596feb6)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.